<a href="https://colab.research.google.com/github/bhanuteja2001/COVID-19-XrayDetection/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [3]:
import pandas as pd
import numpy as np

In [45]:
df = pd.read_csv('/content/drive/MyDrive/covid_image_data/filetrain (1).csv')
df.head()

,Unnamed: 0,filename,file,label
0,0,Image_ex_1.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
1,1,Image_ex_2.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
2,2,Image_ex_3.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
3,3,Image_ex_4.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
4,4,Image_ex_5.jpg,/content/drive/MyDrive/covid_image_data/train/...,0


In [46]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,filename,file,label
0,Image_ex_1.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
1,Image_ex_2.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
2,Image_ex_3.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
3,Image_ex_4.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
4,Image_ex_5.jpg,/content/drive/MyDrive/covid_image_data/train/...,0


In [6]:
df.shape

(2917, 2)

In [7]:
df.label.value_counts()

0    1779
1    1138
Name: label, dtype: int64

In [8]:
df = df.sample(frac = 1).reset_index(drop=True)

In [10]:
Train = df